In [1]:
import sqlite3
from sqlite3 import Error
import requests
import time
import random

In [6]:
####### Database Functions ################
# Create collection table
def create_collection_table(conn):
    cur = conn.cursor()
    cur.execute("""CREATE TABLE IF NOT EXISTS Collection(
                    collection_id text PRIMARY KEY,
                    indicator text,
                    indicator_value text,
                    creation_time text,
                    FOREIGN KEY (indicator) REFERENCES Entrie(indicator)
                );""")
# Create Entries table
def create_entries_table(conn):
    cur = conn.cursor()
    cur.execute("""CREATE TABLE IF NOT EXISTS Entry(
                    indicator,
                    country text,
                    date text,
                    value text
                );""")

# Check the indicator already exist or not
def indicator_already_exist(conn,indicator_data):
    cur = conn.cursor()
    cur.execute(f"""SELECT * FROM Collection 
                    WHERE indicator = "{indicator_data}";
                """)
    result = cur.fetchall()
    if len(result) != 0:
        return True
    else:
        return False

# Insert data to entry table
def insert_entry_data(conn, indicator_data):
    cur = conn.cursor()
    entry = []
    for i in range(50):
        temp = (indicator_data[0], indicator_data[1][i], indicator_data[2][i], indicator_data[3][i])
        entry.append(temp)
    cur.executemany("""INSERT INTO Entry
                (indicator, country, date, value)
                VALUES(?,?,?,?);""", entry)
    conn.commit()
    return cur.lastrowid

# Insert data to collection table
def insert_collection_data(conn, indicator_data):
    cur = conn.cursor()
    cur.execute("""INSERT INTO Collection
                (collection_id, indicator, indicator_value, creation_time)
                VALUES(?,?,?,?);
            """, indicator_data)
    return cur.lastrowid

# Select the data from database
def find_indicator(conn, indicator_data):
    cur = conn.cursor()
    cur.execute(f"""SELECT * FROM Collection 
                    WHERE indicator = "{indicator_data}";
                """)
    result = cur.fetchall()
    return  result

# Select all data from collection table and order by collection_id
def get_all_data(conn):
    sql = """SELECT * FROM Collection ORDER BY collection_id;"""
    cur = conn.cursor()
    cur.execute(sql)
    result = cur.fetchall()
    return result

In [5]:
# random produce collection_id
def random_collection_id():
    d = time.strftime("%Y%m%d%H%M%S",time.localtime(time.time()))
    r = random.sample(d, len(d))
    collection_id = ''
    for i in r:
        collection_id += i
    return collection_id

# 获取url函数
def get_indicator_page(indicator, page):
    return ('http://api.worldbank.org/v2/countries/all/indicators/'+indicator+'?date=2013:2018&format=json&page='+page)

# url = get_indicator_page('NY.GDP.MKTP.CD', '1')

# 获取indicator数据
def get_indicator_data(url):
    r = requests.get(url)
    contents = r.json()
    indicator_value = contents[1][1]['indicator']['value']
    country = []
    date = []
    value = []
    for i in range(50): 
        country.append(contents[1][i]['country']['value'])
        date.append(contents[1][i]['date'])
        value.append(contents[1][i]['value'])
    return indicator_value, country, date, value

In [56]:
conn = sqlite3.connect('data.db') # 连接数据库
create_collection_table(conn)     # 创建表
create_entries_table(conn)

indicator = 'NY.GDP.MKTP.CD'

if indicator_already_exist(conn, indicator):
    result = find_indicator(conn, indicator)
    print({'location': f"/collections/{str(result[0][0])}",
        'collection_id': str(result[0][0]),
        'creation_time': str(result[0][3]),
        'indicator': str(result[0][1])}, 200)  
else:
    url = get_indicator_page(indicator, '1')  # use the indicator to get url
    indicator_value, country, date, value = get_indicator_data(url) # get data from the url
    entries = (indicator, country, date, value) # collect data for entry table
    insert_entry_data(conn, entries)    # insert data to entry table
    creation_time = time.strftime("%Y-%m-%dT%H:%M:%SZ",time.localtime(time.time())) # creation time
    collection_id = random_collection_id()  # generate a collection_id
    collection_data = (collection_id, indicator, indicator_value, creation_time)    # collect data for collection table
    insert_collection_data(conn, collection_data)   # insert data to collection table
    result = find_indicator(conn, indicator)
    q3 = ({
        'location': f"/collections/{str(result[0][0])}",
        'collection_id': str(result[0][0]),
        'creation_time': str(result[0][3]),
        'indicator': str(result[0][1])}, 201)  
        
        
        

In [4]:
def get_all_data():
    conn = sqlite3.connect('Mark.db')
    cur = conn.cursor()
    sql = """SELECT * FROM Collection ORDER BY collection_id;"""
    cur.execute(sql)
    result = cur.fetchall()
    conn.commit()
    conn.close()
    return result

In [67]:
result = get_all_data()
print(result)
print(result[0][0])

[('01230101919320', 'NY.GDP.MKTP.CD', 'GDP (current US$)', '2019-03-21T19:00:13Z'), ('21010229019326', '1.0.HCount.1.90usd', 'Poverty Headcount ($1.90 a day)', '2019-03-21T19:20:26Z')]
01230101919320


In [15]:
collection = '01230101919320'
conn = sqlite3.connect('Mark.db')
cur = conn.cursor()
sql = f"""SELECT indicator FROM Collection WHERE collection_id = "{collection}";"""
cur.execute(sql)
result = cur.fetchall()
a = str(result[0])

In [20]:
print(a)
print(a[2:-3])

('NY.GDP.MKTP.CD',)
NY.GDP.MKTP.CD


NY.GDP.MKTP.CD


In [3]:
def get_entry_data(collection_id):
    conn = sqlite3.connect(db_file)
    cur = conn.cursor()
    cur.execute(f"""SELECT * FROM Entry WHERE id="{collection_id}";""")
    result = cur.fetchall()
    conn.commit()
    conn.close()
    return result

def get_collection_data(collection):
    conn = sqlite3.connect(db_file)
    cur = conn.cursor()
    cur.execute(f"""SELECT * FROM Collection WHERE collection_id="{collection}";""")
    result = cur.fetchall()
    conn.commit()
    conn.close()
    return result
def retrieve_entry_data(collection_id, year, country):
    conn = sqlite3.connect(db_file)
    cur = conn.cursor()
    sql = f"""SELECT * FROM Entry WHERE id="{collection_id}" AND date="{year}" AND country="{country}";"""
    cur.execute(sql)
    result = cur.fetchall()
    conn.commit()
    conn.close()
    return result

In [ ]:
collection_id = 
year = body['year']
country = body['country']
collection_data = get_collection_data(collection_id)
indicator = collection_data[0][1]
entry_data = retrieve_entry_data(indicator, year, country)
print({
    'collection_id': str(collection_id),
    'indicator': str(indicator),
    'country': str(entry_data[0][1]),
    'year': str(entry_data[0][2]),
    'value': str(entry_data[0][3])
})

In [13]:
def get_order_records(collection_id, year, n, order):
    conn = sqlite3.connect('Mark.db')
    cur = conn.cursor()
    cur.execute(f"""SELECT indicator_value FROM Collection WHERE collection_id="{collection_id}";""")
    indicator_value = cur.fetchall()
    if order == 'top':
        cur.execute(f"""SELECT * FROM Entry WHERE id="{collection_id}" AND year="{year}" ORDER BY value DESC Limit {n};""")
    else:
        cur.execute(f"""SELECT * FROM Entry WHERE id="{collection_id}" AND year="{year}" ORDER BY value Limit {n};""")
    entry = cur.fetchall()
    conn.commit()
    conn.close()
    return collection, entry

In [14]:
collection_data, entry_data = get_top_n_records('08293121221020', 5, 'bottom')
return({
    'indicator': str(collection_data[0][1]),
    'indicator_value': str(collection_data[0][2]),
    'entries': [{
        'country': str(entry_data[i][1]),
        'date': str(entry_data[i][2]),
        'value': entry_data[i][3]
    } for i in range(len(entry_data))]
})

{'indicator': 'NY.GDP.MKTP.CD', 'indicator_value': 'GDP (current US$)', 'entries': [{'country': 'Europe & Central Asia', 'date': '2017', 'value': '21444091703392.5'}, {'country': 'Europe & Central Asia', 'date': '2018', 'value': None}, {'country': 'Euro area', 'date': '2013', 'value': '13193290272489.4'}, {'country': 'Euro area', 'date': '2014', 'value': '13478913497853.3'}, {'country': 'Euro area', 'date': '2015', 'value': '11667268254660.3'}]}


In [10]:
db_file = 'data.db'
collection_id = '97110314202021'
collection_data = get_collection_data(collection_id)
entry_data = get_entry_data(collection_id)
'''
print({
        'collection_id': collection_id,
        'indicator': str(collection_data[0][1]),
        'indicator_value': str(collection_data[0][2]),
        'creation_time': str(collection_data[0][3]),
        'entries': [{
            'country': str(entry_data[i][1]),
            'date': str(entry_data[i][2]),
            'value': entry_data[i][3]
        } for i in range(len(entry_data))]})
'''
print(len(entry_data))
print(entry_data[0])
print(entry_data[50])

100
('97110314202021', 'Arab World', '2018', 0.0)
('97110314202021', 'Europe & Central Asia', '2016', 20283925294721.9)


In [19]:
db_file = 'data.db'
collection_id = '19000440822303'
year = '2014'
country = 'Arab World'
collection_data = get_collection_data(collection_id)
entry_data = retrieve_entry_data(collection_id, year, country)
print({
    'collection_id': str(collection_id),
    'indicator': str(collection_data[0][1]),
    'country': str(entry_data[0][1]), 
    'year': str(entry_data[0][2]),
    'value': entry_data[0][3]
})
print(len(entry_data[0]))
print(entry_data)

{'collection_id': '19000440822303', 'indicator': 'AG.PRD.CROP.XD', 'country': 'Arab World', 'year': '2014', 'value': 113.383378082395}
4
[('19000440822303', 'Arab World', '2014', 113.383378082395)]


In [18]:
collection_data = get_collection_data(collection_id)
entry_data = get_entry_data(collection_id)
print(entry_data[:50])
print(entry_data[50:])
'''
print({
        'collection_id': collection_id,
        'indicator': str(collection_data[0][1]),
        'indicator_value': str(collection_data[0][2]),
        'creation_time': str(collection_data[0][3]),
        'entries': [{
            'country': str(entry_data[i][1]),
            'date': str(entry_data[i][2]),
            'value': entry_data[i][3]
        } for i in range(len(entry_data))]})
print(len(entry_data))
'''

[('92242300703103', 'Arab World', '2018', 0.0), ('92242300703103', 'Arab World', '2017', 0.0), ('92242300703103', 'Arab World', '2016', 0.0), ('92242300703103', 'Arab World', '2015', 0.0), ('92242300703103', 'Arab World', '2014', 113.383378082395), ('92242300703103', 'Arab World', '2013', 117.826830053319), ('92242300703103', 'Caribbean small states', '2018', 0.0), ('92242300703103', 'Caribbean small states', '2017', 0.0), ('92242300703103', 'Caribbean small states', '2016', 0.0), ('92242300703103', 'Caribbean small states', '2015', 0.0), ('92242300703103', 'Caribbean small states', '2014', 108.022610986456), ('92242300703103', 'Caribbean small states', '2013', 108.671928210045), ('92242300703103', 'Central Europe and the Baltics', '2018', 0.0), ('92242300703103', 'Central Europe and the Baltics', '2017', 0.0), ('92242300703103', 'Central Europe and the Baltics', '2016', 0.0), ('92242300703103', 'Central Europe and the Baltics', '2015', 0.0), ('92242300703103', 'Central Europe and the 

"\nprint({\n        'collection_id': collection_id,\n        'indicator': str(collection_data[0][1]),\n        'indicator_value': str(collection_data[0][2]),\n        'creation_time': str(collection_data[0][3]),\n        'entries': [{\n            'country': str(entry_data[i][1]),\n            'date': str(entry_data[i][2]),\n            'value': entry_data[i][3]\n        } for i in range(len(entry_data))]})\nprint(len(entry_data))\n"

In [21]:
a = 1464263712669.79
b = 2908394129857.66
if a > b:
    print('dd')
if a < b:
    print('eee')

eee
